In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI迁移：使用自定义训练容器进行自定义图像分类

<table align="left">

  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-image-classification-custom-container.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-image-classification-custom-container.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-custom-image-classification-custom-container.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本笔记本演示了如何使用TensorFlow和Vertex AI SDK创建自定义训练容器来训练自定义图像分类模型。该笔记本还演示了如何将训练好的模型部署到Vertex AI并从中生成预测。

了解更多关于[Migrate to Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用自定义容器和Vertex AI训练来训练一个tensorflow图像分类模型。训练完成后，您还将使用预构建的容器将模型部署到Vertex AI，并对其进行批处理和在线预测。

本教程使用以下谷歌云ML服务和资源：

- *Vertex AI训练*
- *Vertex AI模型注册*
- *Vertex AI批处理预测*
- *Vertex AI终点*

执行的步骤包括：

- *将训练代码打包成python应用程序。*
- *使用Cloud Build和Artifact Registry对训练应用程序进行容器化。*
- *在Vertex AI中创建自定义容器训练作业并运行。*
- *评估从训练作业生成的模型。*
- *在Vertex AI模型注册中为训练模型创建模型资源。*
- *运行Vertex AI批处理预测作业。*
- *将模型资源部署到Vertex AI终点。*
- *在模型资源上运行在线预测作业。*
- *清理创建的资源。*

数据集

本教程使用的数据集是[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/cifar10)中的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/overview)。您使用的数据集版本已内置于TensorFlow中。训练模型预测图像属于十个类别中的哪一个：*飞机*、*汽车*、*鸟*、*猫*、*鹿*、*狗*、*青蛙*、*马*、*船*和*卡车*。

### 成本

该教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud 存储
* Cloud 构建
* 存储库

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[Cloud 存储价格](https://cloud.google.com/storage/pricing)，[Cloud 构建价格](https://cloud.google.com/build/pricing)，[存储库价格](https://cloud.google.com/artifact-registry/pricing) 并使用[定价计算器](https://cloud.google.com/products/calculator/)基于您的预估使用量生成成本估算。

## 安装

安装执行此笔记本所需的包。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage -q

import os
if os.getenv("IS_TESTING"):
    ! apt-get update && apt-get install -y python3-opencv-headless
    ! apt-get install -y libgl1-mesa-dev
    ! pip3 install --upgrade opencv-python-headless -q
    ! pip3 install tensorflow==2.9 -q

### 仅限Colab使用：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[找出项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 设置地区

**可选**：更新'REGION'变量以指定您想要使用的地区。了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源上的名称冲突，您可以为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。Bitte geben Sie den Text ein oder verwenden Sie die Spracherkennung.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud账户

要验证您的Google Cloud账户，请按照您的Jupyter环境的说明操作：

**1. Vertex AI Workbench**
<br>您已经通过验证。

**2. 本地JupyterLab实例**
<br>取消注释并运行以下代码：

In [ ]:
# ! gcloud auth login

取消注释并运行以下代码:

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

请参阅如何将Cloud Storage权限授予您的服务账户的方法，网址为 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时，请运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库

加载在这个笔记本中使用的Vertex AI SDK和其他Python库。

In [ ]:
import base64
import json

import cv2
import numpy as np
import tensorflow as tf
from google.cloud import aiplatform
from tensorflow.keras.datasets import cifar10

### 初始化 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 为培训和预测配置容器

在这一步中，您设置用于创建培训和服务容器的配置参数，如所需的GPU数量和类型（默认情况下使用CPU）以及服务所需的机器类型。


### 设置硬件加速器

您可以为培训和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 来使用支持GPU的容器映像以及分配给虚拟机实例的GPU数量。例如，要使用一个GPU容器映像，并且为每个VM分配了4个 Nvidia Telsa K80 GPU，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，请指定 `(None, None)` 来使用一个在CPU上运行的容器映像。

在[这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)了解更多有关您地区的硬件加速器支持的信息。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (None, None)
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

### 为预测设置预构建容器

设置预构建的Docker容器镜像用于预测。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预构建镜像：

- 要查看可用的预构建镜像列表，请参阅[用于预测的预构建容器文档](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。下面的单元格默认将版本设置为2.9。

In [ ]:
TF = "2-9"

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存。
 - `vCPUs`: 数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下内容不支持用于训练：*

 - `standard`: 2 个 vCPUs
 - `highcpu`: 2, 4 和 8 个 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 包培训应用

在这一步中，您将打包用于在CIFAR10数据集上训练tensorflow图像分类模型的代码，使用您自己创建的自定义容器。

为了使用您自己创建的自定义容器，您需要构建一个Docker文件。首先，您需要为容器组件创建一个目录。

### 包布局

在开始培训之前，您需要查看一个如何为自定义培训工作创建Python包。当解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像的操作环境中的指南。

文件 `trainer/task.py` 是执行自定义培训工作的Python脚本。

*注意：当引用worker池规范中的文件时，目录斜杠会被替换为点(`trainer.task`)，并且会删除文件后缀(`.py`)。*

### 包装配

在下面的单元格中，您将组装训练包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 训练器任务内容

在下一个单元格中，您将编写训练脚本 `task.py` 的内容。这里不会深入讨论脚本的细节。

总结起来，它会：

- 从命令行中获取保存模型数据的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 根据参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

编写Dockerfile内容

容器化您的代码的第一步是创建一个Dockerfile。在Dockerfile中，您包含了运行容器镜像所需的所有命令。在构建过程中，将安装所有软件包并设置一个入口点用于您的训练代码。

在此步骤中，您的Dockerfile将执行以下操作：
1. 从TensorFlow存储库中安装预定义的容器镜像用于深度学习图像。
2. 复制Python训练代码，稍后将显示。
3. 将入口设置为Python训练脚本为`trainer/task.py`。请注意，在ENTRYPOINT命令中省略了`.py`，因为其被隐含。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-9
WORKDIR /root

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

### 启用 Artifact Registry API

您必须为您的项目启用Artifact Registry API服务。

了解有关[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)的更多信息。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

## 创建一个私有的Docker仓库

您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令，在您的区域创建一个新的Docker仓库，描述为“docker仓库”。

2. 运行`gcloud artifacts repositories list`命令来验证您的仓库是否已经创建。

In [ ]:
REPOSITORY = "my-docker-repo"

! gcloud artifacts repositories create {REPOSITORY} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

## 构建训练容器

### 创建一个存储库

接下来，在Artifact Registry中创建一个存储库，用于存储您的训练图像。

在下面为您的存储库和自定义容器设置名称。

In [ ]:
CONTAINER_NAME = "cifar10"

TAG = "latest"
TRAIN_IMAGE = (
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{CONTAINER_NAME}:{TAG}"
)

在Artifact Registry中创建存储库。

### 为培训构建自定义容器

接下来，您使用Cloud Build构建并推送一个docker镜像到创建的存储库。

了解更多关于使用Cloud Build构建和推送Docker镜像的过程。

In [ ]:
%cd custom
!gcloud builds submit --region={REGION} --tag=$TRAIN_IMAGE
%cd ..

### 将培训脚本存储在您的云存储桶中（可选）

接下来，您将培训文件夹打包成一个压缩的tar球，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

## 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练作业，2) 运行这个作业。

有关更多详细信息，请参阅[自定义容器概述]（https://cloud.google.com/vertex-ai/docs/training/containers-overview）。

### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，以下是参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="cifar10_" + UUID, container_uri=TRAIN_IMAGE
)

print(job)

运行自定义训练作业

接下来，您可以通过调用`run`方法来运行自定义作业以启动训练作业，传入以下参数：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作节点的加速器数量。
- `base_output_dir`：Cloud Storage 位置，用于写入模型构件。
- `sync`：是否阻塞直到作业完成。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, UUID)

EPOCHS = 10
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到自定义训练作业完成。

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在，从 Cloud Storage 存储桶中加载它，然后您可以执行诸如评估模型和做预测之类的任务。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径（即 `MODEL_DIR` 指定的路径）作为参数传递给它。

In [ ]:
local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在，您可以通过在测试数据上评估模型来了解模型的表现如何。

### 加载测试数据

为了进行模型评估，您需要使用`tf.keras.datasets`中的`load_data()`方法加载CIFAR10测试（保留）数据。该方法返回一个包含两个元素的元组。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：图像数据和对应的标签。

您不需要训练数据，所以加载时使用`(_, _)`。

在对数据进行评估之前，您需要对其进行预处理：

`x_test`：通过将每个像素除以255来对像素数据进行归一化（重新缩放）。这将使用32位浮点数字代替每个单字节整数像素，范围在0到1之间。

`y_test`：标签目前是标量（稀疏）形式。如果您回顾`trainer/task.py`脚本中的`compile()`步骤，您会发现它是为稀疏标签而编译的。因此我们不需要再做其他处理。

In [ ]:
(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 进行模型评估

评估由自定义作业创建的模型的性能。

In [ ]:
local_model.evaluate(x_test, y_test)

## 图像数据的处理功能

为了将图像传递给预测服务，您需要将压缩（例如JPEG）的图像字节编码为base 64，这样可以使内容在通过网络传输二进制数据时安全免受修改。由于这个部署的模型期望输入数据为原始（未压缩）字节，您需要确保将base 64编码的数据在传递给部署的模型之前转换回原始字节。

为了解决这个问题，您需要定义一个服务功能(`serving_fn`)并将其附加到模型作为预处理步骤。添加`@tf.function`装饰器，以便服务功能与底层模型融合在一起（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容将被base 64解码为Tensorflow字符串（`tf.string`），然后传递给服务功能(`serving_fn`)。服务功能将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`），以满足模型的输入要求：
- `io.decode_jpeg`- 解压JPG图像，返回带有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值改为float 32。
- `image.resize` - 调整图像大小以匹配模型的输入形状。
- `resized / 255.0` - 将像素数据重新缩放（归一化），使其在0到1之间。

此时，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(local_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    local_model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

## 获取serving函数签名

您可以通过重新加载模型到内存，并查询每个图层对应的签名来获取模型的输入和输出图层的签名。

当您将数据发送到HTTP请求包进行预测时，图像数据会以base64编码形式发送，而TF Keras模型只接受numpy输入。您的serving函数会将base64转换为numpy数组。

在进行预测请求时，您需要将请求路由到serving函数而不是模型。因此，在您进行预测请求时，您需要知道serving函数的输入图层名称。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，包括以下参数：

- `display_name`：`Model`资源的可读名称。
- `artifact`：训练模型工件的Cloud Storage位置。
- `serving_container_image_uri`：服务容器镜像。
- `sync`：是否异步或同步执行上传。

如果使用异步方式运行`upload()`方法，您可以随后使用`wait()`方法阻塞直至完成。

了解更多关于[向 Vertex AI 导入模型](https://cloud.google.com/vertex-ai/docs/general/import-model).

In [ ]:
model = aiplatform.Model.upload(
    display_name="cifar10_" + UUID,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

## 进行批量预测

### 获取测试项目

您将使用数据集中保留的测试（留置）部分中的示例作为测试项目。

In [ ]:
test_image_1 = x_test[0]
test_label_1 = y_test[0]
test_image_2 = x_test[1]
test_label_2 = y_test[1]
print(test_image_1.shape)

### 准备请求内容
您将发送CIFAR10图像作为压缩的JPG图像，而不是原始未压缩的字节：

- `cv2.imwrite`：使用OpenCV将未压缩的图像写入磁盘，保存为压缩的JPEG图像。
- 将图像数据从 \[0,1) 范围反归一化到 [0,255)。
- 将32位浮点数值转换为8位无符号整数。

In [ ]:
cv2.imwrite("tmp1.jpg", (test_image_1 * 255).astype(np.uint8))
cv2.imwrite("tmp2.jpg", (test_image_2 * 255).astype(np.uint8))

### 复制测试项目。

为了进行批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
! gsutil cp tmp1.jpg $BUCKET_URI/tmp1.jpg
! gsutil cp tmp2.jpg $BUCKET_URI/tmp2.jpg

test_item_1 = BUCKET_URI + "/tmp1.jpg"
test_item_2 = BUCKET_URI + "/tmp2.jpg"

生成批量输入文件

现在创建一个批量输入文件，将其存储在您的本地Cloud存储桶中。批量输入文件只能是JSONL格式。对于JSONL文件，您为每个数据项（实例）的每行创建一个字典条目。字典包含键/值对：

- `input_name`：基础模型的输入层的名称。
- `'b64'`：指示内容为base64编码的键。
- `content`：压缩的JPG图像字节作为base64编码的字符串。

预测请求中的每个实例是形式为的字典条目：

                        {serving_input: {'b64': content}}

为将图像数据传递给预测服务，您将字节编码为base64。这使内容在通过网络传输二进制数据时免受修改。

- `tf.io.read_file`：将压缩的JPG图像作为原始字节读入内存。
- `base64.b64encode`：将原始字节编码为base64编码的字符串。

了解更多关于[Vertex AI批量预测](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)的信息。

In [ ]:
gcs_input_uri = BUCKET_URI + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    bytes = tf.io.read_file(test_item_1)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")
    bytes = tf.io.read_file(test_item_2)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法发起批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作副本的加速器数量。
- `sync`：如果设置为True，调用将在等待异步批处理作业完成期间阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="cifar10_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测任务完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出桶。您可以调用iter_outputs()方法获取生成结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

进行在线预测

在可以用于提供在线预测之前，您必须将模型部署到一个端点。部署模型会将物理资源与模型关联起来，以便可以使用低延迟提供在线预测。

有关更多详细信息，请参阅[在Vertex AI上获取预测概述](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的易读名称。
- `traffic_split`：在端点上流入此模型的流量百分比，指定为一个或多个键/值对的字典。
如果只有一个模型，则指定为{ "0": 100 }，其中"0"指的是上传的这个模型，100表示100%的流量流向此模型。
如果端点上有现有模型，其流量被分割，则使用`model_id`来指定为{ "0": 百分比，model_id: 百分比，... }，其中`model_id`是部署端点的现有模型的模型ID。这些百分比必须总和为100。
- `machine_type`：用于训练的计算机类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `starting_replica_count`：初始预置的计算实例数量。
- `max_replica_count`：要扩展到的最大计算实例数量。在此教程中，仅预留了一个实例。

In [ ]:
DEPLOYED_NAME = "cifar10-" + UUID

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

获取测试项目

您可以从数据集的测试（保留）部分中取一个示例作为测试项目。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

### 准备请求内容
您发送CIFAR10图像作为压缩的JPG图像，而不是原始的未压缩字节：

- `cv2.imwrite`：使用openCV将未压缩的图像写入磁盘作为压缩的JPEG图像。
 - 将图像数据从\[0,1)范围反归一化为\[0,255)。
 - 将32位浮点值转换为8位无符号整数。
- `tf.io.read_file`：将压缩的JPG图像读取回内存为原始字节。
- `base64.b64encode`：将原始字节编码为base 64编码的字符串。

In [ ]:
cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes_data = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes_data.numpy()).decode("utf-8")

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源，您可以通过向 Endpoint 资源发送预测请求来进行在线预测。

#### 请求

在这个示例中，由于您的测试项目位于一个 Cloud Storage 存储桶中，您需要使用 `tf.io.gfile.Gfile()` 来打开并读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为 base64，这样可以确保在网络上传输二进制数据时内容不会被修改。

每个实例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从 `predict()` 调用返回的是一个Python字典，其中包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类别标签的预测置信度，介于0和1之间。
- `deployed_model_id`：执行预测的已部署 `Model` 资源的 Vertex AI 标识符。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)

print(prediction)

## 下线模型

当你完成预测后，你可以从`Endpoint`资源中下线模型。这将取消所有计算资源，并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

将`delete_bucket`设置为**True**以删除Cloud Storage存储桶。

In [ ]:
delete_bucket = False

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the custom trainig job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

# Delete artifact repository
! gcloud artifacts repositories delete $REPOSITORY --location=$REGION --quiet

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI